In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,13,'Sprint')
session.load()

In [ ]:
df = session.laps.pick_laps(1).pick_drivers('PIA').get_car_data()[['Time','Speed']].add_distance()
df['Time'] = df['Time'].dt.total_seconds()

In [ ]:
def acc_time(df,target_speed):
    
    t1, s1 = df.iloc[df[df.Speed > target_speed].index[0] - 1].Time, df.iloc[df[df.Speed > target_speed].index[0] - 1].Speed
    t2, s2 = df.iloc[df[df.Speed > target_speed].index[0]].Time, df.iloc[df[df.Speed > target_speed].index[0]].Speed

    t_target = t1 + (t2 - t1)*(target_speed - s1)/(s2 - s1)

    acc_time = t_target - df.iloc[df[df.Distance > 0].index[0] - 1].Time

    return round(acc_time,2)

In [ ]:
drivers = session.drivers
driver_dict = {}
for driver in drivers:
    df = session.laps.pick_laps(1).pick_drivers(driver).get_car_data()[['Time','Speed']].add_distance()
    df['Time'] = df['Time'].dt.total_seconds()
    driver_dict[session.get_driver(driver).Abbreviation] = [acc_time(df,100),round(acc_time(df,200)-acc_time(df,100),2)]

In [ ]:
acc_df = pd.DataFrame(driver_dict).T.set_axis(labels=['0-100','100-200'],axis=1)

In [ ]:
acc_df.drop(['COL','GAS'],inplace=True)

In [ ]:
# Sort drivers by 0–100 time
sorted_df = acc_df.sort_values(by='0-100')

drivers = sorted_df.index.tolist()
zero_to_hundred = sorted_df['0-100'].values
hundred_to_two = sorted_df['100-200'].values

# Compute relative deltas
delta_0_100 = zero_to_hundred - zero_to_hundred.min()
delta_100_200 = hundred_to_two - hundred_to_two.min()

gap = 0.04  # Adjust this to control the spacing between bars
y = np.arange(len(drivers))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax.axis('off')

for i in range(len(drivers)):
    color = plotting.get_driver_style(drivers[i],style=['color'],session=session)['color']
    ax.barh(y[i], -delta_0_100[i], left=-gap, color=color)  # Left bar
    ax.barh(y[i], delta_100_200[i], left=gap, color=color)  # Right bar

ax.set_xlim(-0.6,0.6)
ax.invert_yaxis()  # Fastest driver at top

# Add absolute time labels on each bar, and driver name in center
for i, (driver, t1, d1, t2, d2) in enumerate(zip(drivers, zero_to_hundred, delta_0_100, hundred_to_two, delta_100_200)):
    # Left label (0–100 absolute time)
    ax.text(-d1 - 0.05, i, f'{t1:.2f}', va='center', ha='right', fontsize=16, color='w')
    # Right label (100–200 absolute time)
    ax.text(d2 + 0.05, i, f'{t2:.2f}', va='center', ha='left', fontsize=16, color='w')
    # Center driver name
    ax.text(0, i, driver, va='center', ha='center', fontsize=16, fontweight='bold', color='w')

plt.tight_layout()
plt.show()